In [ ]:
# DDGR20，嵌入近似hint，l-noisy < <v,s> < l+noisy
cd framework

In [ ]:
load("../framework/instance_gen.sage")
import numpy as np
import random

In [ ]:
n = 512
m = n
q = 3329
D_s = build_centered_binomial_law(3)
D_e = D_s
A, b, s, dbdd = initialize_from_LWE_instance(DBDD_predict, n, q, m, D_e, D_s)
print(s)
beta, delta = dbdd.estimate_attack()
_ = dbdd.integrate_q_vectors(q, report_every=20)

In [ ]:
# 使用拒绝采样生成[-3*sigma,3*sigma]之间满足高斯分布的随机数
def sample_discrete_gaussian(sigma):
    while True:
        x = random.randint(int(-3 * sigma), int(3 * sigma)) 
        acceptance_prob = math.exp(-x**2 / (2 * sigma**2)) 
        if random.uniform(0, 1) < acceptance_prob: 
            return x
        
def generate_se_eta_sigma_approx_hint(m, n, q, sigma, k):
  V = []
  L = []

  for i in range(k):
    D_e = {-3: 1/64, -2: 6/64, -1: 15/64, 0: 20/64, 1: 15/64, 2: 6/64, 3: 1/64}
    values, probabilities = zip(*D_e.items())
    v = np.array(np.random.choice(values, size=m+n, p=probabilities))
    e = sample_discrete_gaussian(sigma)
    l = dbdd.leak(v) + e
    V.append(v)
    L.append(l)
  print("L",L)
  return V,L

In [ ]:
nph_LWE_80 = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
dis_LWE_80_our = [53.58, 52.78, 51.92, 51.02, 50.19, 49.05, 48.41, 47, 46.58, 45.34, 44.74, 43.55, 42.96, 41.88, 40.73, 41.3, 38.99]
nph_Kyber128 = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650]
dis_Kyber128_our = [20.42, 20.36, 20.1, 19.76, 19.44, 19, 18.61, 18.1, 17.86, 17.42, 17.03, 16.52, 16.19, 15.79, 15.51, 14.97, 14.64, 14.13, 13.95, 13.44, 13.09, 12.89, 12.47, 12.04, 11.69, 11.48, 11.19, 10.82, 10.48, 10.42, 9.98, 9.83, 9.46, 9.31, 9.05, 8.76, 8.51, 8.18, 8.03, 7.63, 7.32, 7.15, 6.58, 6.33, 6.08, 5.37, 4.91, 4.41, 3.6, 2.97, 2.66, 1.84, 1.8, 1.08, 1.15, 0.79, 0.58, 0.37, 0.11, 0.1, 0.14, 0.06, 0.04, 0.03, 0.01, 0.02]
nph_Kyber256 = [0, 40, 80, 120, 160, 200, 240, 280, 320, 360, 400, 440, 480, 520, 560, 600, 640, 680, 720, 760, 800, 840, 880, 920, 960, 1000, 1040, 1080, 1120, 1160]
dis_Kyber256_our = [26.91, 26.39, 25.45, 24.6, 23.65, 22.08, 21.07, 20.21, 18.78, 18.23, 16.86, 16.03, 15.29, 14.36, 13.62, 13.1, 11.79, 11.05, 10.44, 9.17, 6.05, 4.61, 1.08, 0.97, 0, 0, 0, 0, 0, 0]
nph_Kyber512 = [0, 40, 80, 120, 160, 200, 240, 280, 320, 360, 400, 440, 480, 520, 560, 600, 640, 680, 720, 760, 800, 840, 880, 920, 960, 1000, 1040, 1080, 1120, 1160, 1200, 1240, 1280, 1320, 1360, 1400, 1440, 1480, 1520, 1560, 1600, 1640, 1680, 1720, 1760, 1800]
dis_Kyber512_our = [39.9, 39.78, 39.31, 38.57, 37.41, 36.68, 35.87, 35.06, 34.16, 33.6, 32.62, 31.53, 30.82, 30.24, 29.3, 28.57, 27.16, 26.57, 25.88, 24.7, 24.55, 23.62, 22.97, 22.41, 21.66, 21.16, 20.52, 19.82, 19.15, 18.44, 18.04, 17.73, 17.05, 16.21, 15.82, 14.84, 14.19, 13.01, 12.17, 11.49, 8.79, 7.3, 3.78, 2.62, 0.32, 0.05]


num_hint = 1800
sigma = 2
V, L = generate_se_eta_sigma_approx_hint(m, n, q, sigma, num_hint)
BETA = []
index = 0
for j in range(num_hint):
    if j == nph_Kyber512[index]:
        index += 1
        _ = dbdd.integrate_q_vectors(q, report_every=20)
        st = dis_Kyber512_our[index]/dis_Kyber512_our[0]
        beta, delta = dbdd.estimate_attack_SMY(st)
        print("beta: ", beta)
        BETA.append(beta)
    print("the ",j+1,"-th secret error approx hint")
    _ = dbdd.integrate_approx_hint(vec(V[j]), L[j], sigma**2, aposteriori=False)
print(BETA)